In [ ]:
import pandas as pd
import json
from functools import reduce
from collections import defaultdict

In [ ]:
NLP_RESULTS = '/Users/nathansuberi/Documents/GitHub/ResourceWatchCode/Analytic Tools/climatewatch_ndc_nlp/data/nlp_results.txt'
COUNTRY_CATEGORIES = '/Users/nathansuberi/Documents/GitHub/ResourceWatchCode/Analytic Tools/climatewatch_ndc_nlp/data/country_grouping.csv'

In [ ]:
categories = pd.read_csv(COUNTRY_CATEGORIES).set_index('countryCode')
with open(NLP_RESULTS, 'r') as src:
    nlp_results = json.load(src)

In [ ]:
nlp_results

In [ ]:
categories = categories.drop('countryName', axis=1)

In [ ]:
def create_summaries(agg, cntry, nlp, group, groupings):
    try:
        cntry_group = groupings.loc[cntry, group]
        for word, count in nlp[cntry].items():
            agg[cntry_group][word] += count
    except:
        pass
    return agg
    
def dd_wrapper():
    return defaultdict(int)
    
def flatten(dd):
    dd = dict(dd)
    for key, _dd in dd.items():
        dd[key] = dict(_dd)
    return dd
        
topics_by_income = flatten(reduce(lambda agg, country: create_summaries(agg, 
                                               country, 
                                               nlp_results, 
                                               'byIncomeTXT', 
                                               categories), 
                                          nlp_results, 
                                          defaultdict(dd_wrapper)))

In [ ]:
grouped_results = {}
for cat in categories.columns:
    grouped_results[cat] = flatten(reduce(lambda agg, country: create_summaries(agg, 
                                               country, 
                                               nlp_results, 
                                               cat, 
                                               categories), 
                                          nlp_results, 
                                          defaultdict(dd_wrapper)))

In [ ]:
with open('grouped_results.txt', 'w') as dst:
    dst.write(json.dumps(grouped_results))